Pink Buckle Barrel Racing - Stallion Analysis


In [ ]:
#Asses data

df = pd.read_csv("PB-Futurity-2019-1st-Go.csv",  header = None) # read in the CSV usign the first 4 rows as header and first col as identifier
df.head(30)

Clearly there is some cleaning to be done/ we need to tidy the data, upon inspection of the PDFs there were irregularities in the format results were reported in each year. 

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, fnmatch

In [14]:
def expandcolumns_goround(file):
    '''
    INPUT string of path to CSV file converted from PDF of go round results from Pink Buckle Futurity
    Futurity result formatting varies slightly year to year. 
    OUTPUT dataframe containing information relative to a stallion's offspring performance
    '''
    df = pd.read_csv(file,  header = None)
    f = file
    fvals = f.split(sep = '.')
    fvals = fvals[0]
    fvals=fvals.split(sep = '-')
    sex_list = ['Gelding','Mare','Stallion','M','G','S']
    sex_dict = {'G)':'Gelding','M)':'Mare','S)':'Stallion', ' M)':'Mare', 'G':'Gelding'}
    if fvals[2] == '2020':
    #break column 2 into its own data frame, then create tidy data by ensuring each type of "observation" has its own column
        col2 = df.iloc[:,1]
        col2 = col2.str.split (pat = '\r',expand = True)
        dds = (col2.iloc[:,3]).str.split(pat=', ',expand = True)
        acs = ((col2.iloc[:,1]).str.split(pat=' ',expand = True))

        col2 = df.iloc[:,1]
        col2 = col2.str.split (pat = '\r',expand = True)
        dds = (col2.iloc[:,3]).str.split(pat=', ',expand = True)
        dds_header = dds.iloc[0]# Save first row for headers
        dds = dds[1:]
        dds.columns = dds_header
        acs = ((col2.iloc[:,1]).str.split(pat=' ',expand = True))
        acs_header = acs.iloc[0]# acs.iloc[0]# Save first row for headers
        acs.columns = acs_header
        acs=acs[1:]
        # remove columns that have been expaned and concatenate dataframes
        col2 = col2.drop(columns = [1,3])
        col2_header = col2.iloc[0]# Save first row for headers
        col2.columns = col2_header
        col2 = col2[1:]
         # expand column 3 in df the same way as 2 but assign header values
        col3 = (df.iloc[:,2]).str.split(pat='\r', expand = True)

        col3_header = ['Owner1','Owner1_location','Owner2','Owner2_location', 'Rider']
        col3.columns = col3_header
        col3=col3[1:]
        col3=col3['Rider']# We are only using this information for the analysis

        #col4 = (df.iloc[:,3]).str.split(pat='\r', expand = True)
       # col4_header = col4.iloc[0]
       # col4.columns = ['Breeder', 'overflow', 'BreederLoc1', 'BreederLoc1', 'overflow2'] #considering adding overflow to the original columns. IDK how that is going to work. 
       # col4=col4[1:]
        #drop the columns that have been expanded & those not used
        df2 = df.drop(columns = [1,2,3,4,5,6,7])
        df2_header = df2.iloc[0] #promote the first row to headers 
        df2= df2[1:]
        df2.columns = df2_header
        df = pd.concat([col2,dds,acs,col3,df2], axis=1)#concatenate the expanded data
        df['Sex'] = np.where(df.iloc[:,7].notnull(), df.iloc[:,7],df['Sex'])#some of the sex information got pushed into another column using the " " as a delimiter. This moves it back
        #Some entrants did not list color and the sex information was stored in the color field
       
        df['Sex']=np.where(df['Color,'].isin(sex_list), df['Color,'], df['Sex'])
        df = df.rename(columns = {'Color,':'Color'})
        df = df.rename(columns = {'Age,':'Age'})

    elif fvals[2] == '2018' :
        col2 = df.iloc[:,1]
        col2 = col2.str.split (pat = '\r',expand = True)
        horse_age_color_sex = col2.iloc[:,0].str.split(pat = '(', expand = True)
        col2 = col2.drop(columns = [0])
        age_color_sex = horse_age_color_sex.iloc[:,1].str.split(pat='‐', expand = True)
        age_color_sex = age_color_sex[1:]
        horse = horse_age_color_sex.drop(columns = 1)
        horse_head = ['Horse']
        horse=horse[1:]
        horse.columns = horse_head
        acs_header = ['Age', 'Color', 'Sex']
        age_color_sex = age_color_sex.drop(columns = 3)
        age_color_sex.columns = acs_header
        #replace any erroneous sex data that is be in the color column
       # age_color_sex['Sex']=np.where(age_color_sex['Color'].isin(sex_list), age_color_sex['Color'], age_color_sex['Sex'])
        sex_dict = {'G)':'Gelding','M)':'Mare','S)':'Stallion', ' M)':'Mare', 'Bay':'Gelding'} # replace abbreviation with word and brute force change one erroneous point of Bay
        age_color_sex = age_color_sex.replace(to_replace = sex_dict)
        col2_header = col2.iloc[0]# Save first row for headers
        col2.columns = col2_header
        col2 = col2[1:]
       
        col3 = (df.iloc[:,3]).str.split(pat='\r', expand = True)
        col3=col3.drop(columns = 1)
        col3_header = ['Rider']
        col3.columns = col3_header
        col3=col3[1:]
        df2 = df.drop(columns = [1,2,3,4,5,6,])
        df2_header = df2.iloc[0] #promote the first row to headers 
        df2= df2[1:]
        df2.columns = df2_header
        df = pd.concat([horse,age_color_sex,col2,col3,df2], axis=1)#concatenate the expanded data
    else:
        col2 = df.iloc[:,1]# remove the second column to expand
        col2 = col2.str.split(pat = '\r',expand = True)
        dds = col2[2].str.split(pat = ',',expand = True) # split dam and dams sire into separate columns
        dds_header = dds.iloc[0]# Save first row for headers
        dds=dds[1:]
        dds.columns = dds_header
        horse_age_color_sex = col2.iloc[:,0].str.split(pat = '(', expand = True)
        col2 = col2.drop(columns = [0,2])
        horse = horse_age_color_sex.drop(columns = 1)
        horse_head = ['Horse']
        horse=horse[1:]
        horse.columns = horse_head
        age_color_sex = horse_age_color_sex[1]#further break up column 2
        age_color_sex = age_color_sex.str.split(pat='-', expand = True)
        age_color_sex = age_color_sex[1:]
        acs_header = ['Age', 'Color', 'Sex']
        age_color_sex.columns = acs_header
        #replace any erroneous sex data that is be in the color column
        age_color_sex['Sex']=np.where(age_color_sex['Color'].isin(sex_list), age_color_sex['Color'], age_color_sex['Sex'])
      
        age_color_sex = age_color_sex.replace(to_replace = sex_dict)
        
        col2_header = col2.iloc[0]
        col2.columns = col2_header
        col2 = col2[1:]
        col3 = (df.iloc[:,2]).str.split(pat='\r', expand = True) # Expand column 3
        col3_header = ['Rider']
        # rider information was stored in the owner column in 2019 if the owner rode the horse, moving to the rider column
        col3[2] = np.where(col3.iloc[:,2].str.contains(pat = ','), col3.iloc[:,0],col3[2]) #moves owner to rider column if address carried over
        col3[2] = np.where(col3.iloc[:,2].isnull(), col3.iloc[:,0],col3[2]) #moves owner to rider column if there is no data in rider column
        
        col3 = col3.drop(columns = [0,1])
        col3.columns = col3_header
        col3=col3[1:]        
        df2 = df.drop(columns = [1,2,3,4,5,6,])
        df2_header = df2.iloc[0] #promote the first row to headers 
        df2= df2[1:]
        df2.columns = df2_header
        df = pd.concat([horse,age_color_sex,col2,col3,df2], axis=1)#concatenate the expanded data
   
    #remove header information carried over from converting each pafe of PDF files to CSV
    df.drop(df.loc[df['Sire']=='Sire'].index, inplace=True)
    # reset the index then rename index as the round results for that file
    df.reset_index(inplace=True)
       #rename index from 
    f = file
    fvals = f.split(sep = '.')
    fvals = fvals[0]
    fvals=fvals.split(sep = '-')
    indexstring = ' '.join(fvals[3:])
    indexstring = indexstring + ' Place'
    df = df.rename(columns = {'index':indexstring})

    return df

Import all data throg

In [15]:
go1_18 = expandcolumns_goround('PB-Futurity-2018-1st-Go.csv').drop(columns = ['#'])
go2_18 = expandcolumns_goround('PB-Futurity-2018-2nd-Go.csv').drop(columns = ['#'])
pbf_18 = pd.merge(go1_18,go2_18, how = "outer")
#am = pd.merge(go1_18_am,go2_18_am)
#pbf_18=pd.merge(pbf_18, am,how= "outer")
#Eddie S nson is not a unique sire... Lets change that back to Eddie StinJson
replace = {'Eddie S nson':'Eddie Stinson','BHR Frenchie Socks':'BHR Frenchies Socks'}

pbf_18.replace(to_replace = replace, inplace = True)
#change time, placing and year to integers
pbf_18 = pbf_18.astype({'Horse' : 'string', 'Age':'float64', 'Color' : 'string', 'Sex' : 'string', 'Sire' : 'string', 'Dam' : 'string',
       "Dam's Sire":'string', 'Rider' : 'string', '1st Go': 'float64', '2nd Go': 'float64'})
pbf_18['Sire'].str.title()
pbf_18

,1st Go Place,Horse,Age,Color,Sex,Sire,Dam,Dam's Sire,Rider,1st Go,2nd Go Place,2nd Go
0,1.0,French Goodbye,2014.0,Palomino,Gelding,The Goodbye Lane,Two Fools Gold,Frenchmans Guy,Vauna Walker,16.567,5.0,16.936
1,2.0,Jets Letters Ta Abby,2014.0,Sorrel,Gelding,Blazin Jetolena,Real Famous Abby,Dash Ta Fame,Ashley Schafer,16.845,7.0,16.997
2,3.0,Hes Furious,2014.0,Gelding,Gelding,The Goodbye Lane,Furyx,Calyx,Timi Lickley,16.988,8.0,16.998
3,4.0,Jets Heart N Soul,2014.0,Chestnut,Gelding,Blazin Jetolena,Mistys Dash of Fame,Dash Ta Fame,Jolene Montgomery,16.988,2.0,16.818
4,5.0,Our First Goodbye,2013.0,Brown,Mare,The Goodbye Lane,Jesse James Agent,Jesse James Jr,Marcie Wilson,17.005,66.0,22.455
...,...,...,...,...,...,...,...,...,...,...,...,...
72,77.0,Patches O Houlihan,2013.0,Gelding,Mare,PC Redwood Manny,Patchs April Gem,Tiny Patch,Kelsey Hayden,30.174,52.0,18.380
73,78.0,HP Dash Ta Fiesta *PBS,2013.0,Sorrel,Stallion,Dash Ta Fame,Firewater Fiesta,Fire Water Flit,Ryann Pedone,999.000,30.0,17.459
74,79.0,Dats A Gin Fizz,2013.0,Chestnut,Mare,Dats A Frenchman,Regal Ginnie,Regal Legacy,Kelli Barichello,999.000,79.0,999.000
75,80.0,Flit To Be Famous,2014.0,Sorrel,Gelding,Eddie Stinson,Pepoleo Flit,Fire Water Flit,Chelsea Jepperson,999.000,80.0,999.000


In [16]:
go1_19 = expandcolumns_goround('PB-Futurity-2019-1st-Go.csv')
go2_19 = expandcolumns_goround('PB-Futurity-2019-2nd-Go.csv').drop(columns = ['Horse', 'Age', 'Color', 'Sex', 'Sire', 'Rider'])


pbf_19 = pd.merge(go1_19,go2_19, how ="outer")

pbf_19.dtypes
#change time, placing and year to integers
pbf_19 = pbf_19.astype({'Horse' : 'string', 'Age':'int64', 'Color' : 'string', 'Sex' : 'string', 'Sire' : 'string',
        'Rider' : 'string', '1st Go': 'float64', '2nd Go': 'float64'})

In [49]:
go1_20 = expandcolumns_goround('PB-Futurity-2020-1st-Go.csv')
go2_20 = expandcolumns_goround('PB-Futurity-2020-2nd-Go.csv').drop(columns = ['Horse', 'Age', 'Color', 'Sex','Rider', 'Sire', 'Dam', "Dam's Sire"])
pbf_20 = pd.merge(go1_20,go2_20, on ='#').drop(columns = ['None_x','None_y'])
#change time, placing and year to integers
pbf_20 = pbf_20.astype({'Horse' : 'string', 'Age':'float64', 'Color' : 'string', 'Sex' : 'string', 'Sire' : 'string', 'Dam' : 'string',
       "Dam's Sire":'string', 'Rider' : 'string', '1st Go': 'float64', '2nd Go': 'float64'})


,1st Go Place,Horse,Sire,Dam,Dam's Sire,Age,Color,Sex,Rider,#,1st Go,2nd Go Place,2nd Go
0,1,VF Cream Rises,Eddie Stinson,Curiocity Corners,Silver Lucky Buck,2016.0,Sorrel,Mare,Kelsey Treharne,13,17.067,183,30.653
1,2,KN Snap Back,Eddie Stinson,KN Hanks Design,Designer Red,2016.0,Mare,Mare,Caroline Boucher,186,17.070,3,17.092
2,3,The Right Version,Winners Version,The Right Life,Dash To Chivato,2015.0,Gray,Mare,McKinlee Kellett,163,17.102,27,17.511
3,4,Makana,Slick By Design,Rods Last Ladybug,Sharp Rodney,2015.0,Sorrel,Mare,Lindsey McLeod,18,17.125,12,17.259
4,5,CBC Speeding Goodbye,The Goodbye Lane,Speeding Streaker,Mitbeastreaker,2016.0,Sorrel,Gelding,Vauna Walker,89,17.151,6,17.131
5,6,MJ Night Train Lane,The Goodbye Lane,Moment of Pursuit,Dash Ta Fame,2015.0,Brown,Gelding,Marcie Wilson,142,17.180,10,17.243
6,7,MJ Lola Lane,The Goodbye Lane,Pappys Fools Ta Fame,Easy Illusion,2015.0,Chesnut,Mare,Marcie Wilson,83,17.208,13,17.284
7,8,Boonstreakincolours,Streakin Boon Dox,Frenchmans Colours,Hot Colours,2015.0,Bay,Gelding,Ashley Schafer,1,17.214,155,22.304
8,9,Slicks Famous Design,Slick By Design,Dasharuth,Dash Ta Fame,2016.0,Black,Mare,Jana Bean,148,17.235,15,17.345
9,10,Blazing With My Dude,Blazin Jetolena,Dasher Dude,Texas High Dasher,2015.0,Sorrel,Mare,Sue Smith,177,17.274,48,17.687


In [20]:
'''Payouts are based on Divisions which are a set time period off the fastest time.  This is called a D.  
We will assign a D to each horse per run for each year.  Later this will be helpful to see if any particular sire's offspring
are more likely to be in any particular division, we will also allocate the first 20 horses in both Ds as money winners per the curtent payout format'''

def AssignDs(df, colstring1, new_col_name):
    '''
    assign D's based on division split, and give the first 20 horses in each D the status of "Money Winner"
    Input: df = Dataframe, 
    colstring1 = String - the time column on the data frame to be placing D's in.
    new_col_name = String - the column name the Ds will be placed in
    output: adds column placeing horse in 1D or 2D
    '''
    D_split = 0.75 # D split for the PB futurity and Derby
    df[new_col_name]= np.where(df[colstring1] > (df[colstring1].min()+D_split), 2, 1)
    
    place_col_name = 'colstring1'+' Place' #create a value to pull the money winners by placing
    df['Money_Winner'] = np.where(df['place_col_name']<= 20, 1,0) #calls out money winners in 1D
    df.sort_values('colstring1', inplace = True) #sort so we can call the 2D winner index  
    two_d = df[df['new_col_name'] > 1].index[0] #get the next 20 places, 
    df.iloc[two_d:(two_d+19),df.columns.get_loc('Money_Winner')] = 1 #assign 1 for money winner
    return df
    

In [50]:
pbf_18 = AssignDs(pbf_18, '1st Go', '1stGoD')
pbf_19 = AssignDs(pbf_19, '1st Go', '1stGoD')
pbf_20 = AssignDs(pbf_20, '1st Go', '1stGoD')
pbf_18 = AssignDs(pbf_18, '2nd Go', '2ndGoD')
pbf_19 = AssignDs(pbf_19, '2nd Go', '2ndGoD')
pbf_20 = AssignDs(pbf_20, '2nd Go', '2ndGoD')

,1st Go Place,Horse,Age,Color,Sex,Sire,Dam,Dam's Sire,Rider,1st Go,2nd Go Place,2nd Go,1stGoD,2ndGoD,Money_Winner,1D_Money_Winner
0,1.0,French Goodbye,2014.0,Palomino,Gelding,The Goodbye Lane,Two Fools Gold,Frenchmans Guy,Vauna Walker,16.567,5.0,16.936,1,1,1,1
1,2.0,Jets Letters Ta Abby,2014.0,Sorrel,Gelding,Blazin Jetolena,Real Famous Abby,Dash Ta Fame,Ashley Schafer,16.845,7.0,16.997,1,1,1,1
2,3.0,Hes Furious,2014.0,Gelding,Gelding,The Goodbye Lane,Furyx,Calyx,Timi Lickley,16.988,8.0,16.998,1,1,1,1
3,4.0,Jets Heart N Soul,2014.0,Chestnut,Gelding,Blazin Jetolena,Mistys Dash of Fame,Dash Ta Fame,Jolene Montgomery,16.988,2.0,16.818,1,1,1,1
4,5.0,Our First Goodbye,2013.0,Brown,Mare,The Goodbye Lane,Jesse James Agent,Jesse James Jr,Marcie Wilson,17.005,66.0,22.455,1,2,1,1
5,6.0,High Cotton Lane,2014.0,Chestnut,Stallion,The Goodbye Lane,HMB Soon Tobe Famous,Dash Ta Fame,Paige Jones,17.021,3.0,16.829,1,1,1,1
6,7.0,NB Ladybird,2014.0,Gray,Mare,Shawne Bug Leo,TR Dashing Badger,Mr Illuminator,Kenna Squires,17.050,11.0,17.087,1,1,1,1
7,8.0,Streakin French Star,2014.0,Buckskin,Gelding,Streakin Boon Dox,French Stars,Frenchmans Guy,Ashley Schafer,17.091,14.0,17.166,1,1,1,1
8,9.0,Positively Streakin,2014.0,Bay Roan,Mare,Streakin Boon Dox,Positively A Coyote,Coyote Colonel,Ashley Schafer,17.117,15.0,17.208,1,1,1,1
9,10.0,Mr Elegant Shawne,2014.0,Sorrel,Mare,Shawne Bug Leo,Elegant Seis,Tres Seis,Jacque Woolman,17.153,10.0,17.076,1,1,1,1


In [51]:
#Concatenate all 3 years results into one dataframe
pbf = pd.concat([pbf_18, pbf_19,pbf_20])
pbf['Sire'] = pbf['Sire'].str.upper() # move all words to upper case to join with 2021 stud fee info from Qstallions.com
pbf.head()

,1st Go Place,Horse,Age,Color,Sex,Sire,Dam,Dam's Sire,Rider,1st Go,2nd Go Place,2nd Go,1stGoD,2ndGoD,Money_Winner,1D_Money_Winner,#
0,1.0,French Goodbye,2014.0,Palomino,Gelding,THE GOODBYE LANE,Two Fools Gold,Frenchmans Guy,Vauna Walker,16.567,5.0,16.936,1,1,1.0,1.0,NaN
1,2.0,Jets Letters Ta Abby,2014.0,Sorrel,Gelding,BLAZIN JETOLENA,Real Famous Abby,Dash Ta Fame,Ashley Schafer,16.845,7.0,16.997,1,1,1.0,1.0,NaN
2,3.0,Hes Furious,2014.0,Gelding,Gelding,THE GOODBYE LANE,Furyx,Calyx,Timi Lickley,16.988,8.0,16.998,1,1,1.0,1.0,NaN
3,4.0,Jets Heart N Soul,2014.0,Chestnut,Gelding,BLAZIN JETOLENA,Mistys Dash of Fame,Dash Ta Fame,Jolene Montgomery,16.988,2.0,16.818,1,1,1.0,1.0,NaN
4,5.0,Our First Goodbye,2013.0,Brown,Mare,THE GOODBYE LANE,Jesse James Agent,Jesse James Jr,Marcie Wilson,17.005,66.0,22.455,1,2,1.0,1.0,NaN


In [75]:
df = pbf.pivot_table(
    index = ['Sire','Horse'],
    values = ['1st Go','2nd Go','1st Go Place', '2nd Go Place','1stGoD','2ndGoD','Money_Winner'] 
)


df['1D Horse']=(df[df == 1].sum(axis = 1)) #assigns a value of 1 for each run a horse had <.75 seconds off the fastest time in their year

#df18 = bf_18.groupby(by = ['Sire']).mean()
#pd.pivot_table(df,index=["Manager","Rep"],values=["Price"],aggfunc=[np.mean,len])
#'

'''Tipped barrels were considered no time at this event and assigned a time of 999.00 for the round. 
Including this in the average will unfairly weight the times in favor of those whose offspring did not tip a barrel.
This should not fault the horse as it could easily be rider error.  Being the results have no way of indicating a horse did not make a run 
and took a no time or if they tipped all 3 barrels, no times were not included in the average. 
    '''
    
df['Tipped'] = (df[df == 999.000].sum(axis = 1))

df.replace(to_replace = {'Tipped': {999.0 : 1, 1998.0 : 2}}, inplace = True)
tipcount = df.groupby(['Sire']).sum()
oneDcount = df.groupby(['Sire'])




(df.replace(to_replace =  {999.0 :np.nan}, inplace = True)) #remove 999.00 to become nan so they aren't included in the average time
df = df.dropna(subset=['1st Go', '2nd Go']) #remove any horse that had no time for both go's

siredata['total_offspring'] = df.groupby(['Sire']).size()

siredata.head(30)

,1st Go,1st Go Place,1stGoD,2nd Go,2nd Go Place,2ndGoD,Money_Winner,1D Horse,Tipped,total_offspring
Sire,,,,,,,,,,
A SMOOTH GUY,18.871968,84.806452,1.741935,19.048581,88.451613,1.806452,0.285714,0.516129,0.0,31
BHR FRENCHIES SOCKS,19.067304,81.434783,1.760870,18.868304,75.130435,1.630435,0.200000,0.652174,0.0,23
BLAZIN JETOLENA,18.612400,66.628571,1.571429,19.744057,82.114286,1.714286,0.571429,0.857143,0.0,35
BORN TA BE FAMOUS,20.107200,96.700000,1.800000,19.592600,84.900000,1.900000,0.000000,0.300000,0.0,10
CARRIZZO,22.609000,139.000000,2.000000,28.651000,156.000000,2.000000,NaN,0.000000,0.0,1
CONFEDERATE LEADER,17.914000,73.000000,2.000000,18.286000,101.000000,2.000000,NaN,0.000000,0.0,1
CORONA CARTEL,17.963000,77.000000,2.000000,17.655000,49.000000,1.000000,NaN,1.000000,0.0,1
DASH TA FAME,17.601000,25.000000,2.000000,17.846000,43.000000,2.000000,1.000000,1.000000,0.0,1
DASHIN DYNAMO,18.779000,88.833333,1.833333,17.960500,74.333333,1.833333,NaN,0.333333,0.0,6


In [59]:
#move totals to the dataframe of average times
siredata['Total_no-times'] = tipcount['Tipped']
siredata['1D Horse'] = tipcount['1D Horse']
siredata['Money_Winner'] = tipcount['Money_Winner']
#siredata['Remove tipped'] = 
siredata.head(50)
siredata['Adj Average_Time'] = ((df.groupby(['Sire']).mean()).loc[:,['1st Go','2nd Go']]).sum(axis = 1) #caluclates the sum of the (average 1st Go and second go for all offspring 
siredata['Adj Average_placing'] = ((df.groupby(['Sire']).mean()).loc[:,['1st Go Place','2nd Go Place']]).mean(axis = 1)

siredata['Average_Time'] = (siredata.loc[:,['1st Go','2nd Go']]).sum(axis = 1) #caluclates the sum of the (average 1st Go and second go for all offspring 
siredata.rename(columns = {'Tipped':'Normalized_penalties'}, inplace = True)
siredata['Average_placing'] = (siredata.loc[:,['1st Go Place','2nd Go Place']]).mean(axis = 1)
siredata.sort_values(by = ['Adj Average_Time', 'Total_no-times'] ,ascending=True, inplace=True)

#siredata['No-time runs (%)'].multiply(100, axis = 1)
#filter out tipped barrels & replace with an average.  Then view data
siredata['pct1D']= 100*(siredata['1D Horse']/siredata['total_offspring'])
siredata['pctMoneyWinners'] = 100*(siredata['Money_Winner']/siredata['total_offspring'])
siredata

,1st Go,1st Go Place,1stGoD,2nd Go,2nd Go Place,2ndGoD,Money_Winner,1D Horse,Normalized_penalties,total_offspring,Total_no-times,Total Money Winners,Adj Average_Time,Adj Average_placing,Average_Time,Average_placing,pct1D,pctMoneyWinners
Sire,,,,,,,,,,,,,,,,,,
DATS A FRENCHMAN,671.942333,114.666667,2.000000,671.853667,102.666667,1.666667,0.0,1.0,1.333333,3,4.0,0.0,35.388000,50.000000,1343.796000,108.666667,33.333333,0.000000
DASH TA FAME,508.300500,51.500000,2.000000,17.652500,36.500000,1.500000,1.0,2.0,0.500000,2,1.0,1.0,35.447000,34.000000,525.953000,44.000000,100.000000,0.500000
WINNERS VERSION,508.335000,127.500000,1.750000,508.453250,134.000000,1.750000,0.0,2.0,1.000000,4,4.0,0.0,35.576500,64.250000,1016.788250,130.750000,50.000000,0.000000
CORONA CARTEL,17.963000,77.000000,2.000000,17.655000,49.000000,1.000000,0.0,1.0,0.000000,1,0.0,0.0,35.618000,63.000000,35.618000,63.000000,100.000000,0.000000
GUYS EYE PAINT,17.875000,75.000000,2.000000,18.038000,94.000000,2.000000,0.0,0.0,0.000000,1,0.0,0.0,35.913000,84.500000,35.913000,84.500000,0.000000,0.000000
JL SIROCCO,17.800500,64.000000,1.500000,18.376000,89.500000,1.500000,0.0,2.0,0.000000,2,0.0,0.0,36.176500,76.750000,36.176500,76.750000,100.000000,0.000000
CONFEDERATE LEADER,17.914000,73.000000,2.000000,18.286000,101.000000,2.000000,0.0,0.0,0.000000,1,0.0,0.0,36.200000,87.000000,36.200000,87.000000,0.000000,0.000000
STREAKIN BOON DOX,344.664250,79.000000,1.416667,427.331917,108.166667,1.750000,3.0,13.0,0.750000,12,9.0,3.0,36.303429,41.642857,771.996167,93.583333,108.333333,0.250000
NO PRESSURE ON ME,17.972500,78.500000,1.500000,18.346333,77.500000,1.666667,0.0,5.0,0.000000,6,0.0,0.0,36.318833,78.000000,36.318833,78.000000,83.333333,0.000000


In [ ]:
df_sires = pd.read_csv('PB_Sires_2021_Fees.csv')
sire_fees=df_sires.iloc[:,[0,5]] #drop columns with information that are not to be used in our anaylisis
df2 = pd.merge(siredata, sire_fees, on = 'Sire', how = 'outer')
df2.columns

In [ ]:
df2 = df2.drop(columns = (['1st Go', '1st Go Place', '1stGoD', '2nd Go', '2nd Go Place',
       '2ndGoD']))

In [ ]:
#display(df2)
no_entries = (df2['1D Horse'].isna()).sum() # count the number of 2021 sires who have had not had offspring compete
no_entries
ineligible = (df2['Fee'].isna()).sum()
print('{} Sires have not had any offspring run in the Futurity, {} sires are no longer eligible as of the 2021 breeding season'.format(no_entries, ineligible))
df2.head(62)

In [ ]:
df2 = df2.dropna(subset=['Fee', '1D Horse'])
df2.shape

In [ ]:
df2.sort_values(by = ['total_offspring'],ascending=False)
df2.sort_values(by = ['pct1D'],ascending=False)

df2.sort_values(by = ['Adj Average_placing'],ascending=True)

In [ ]:
df2.shape

In [60]:
#create plots of avg time by sire (box) 

In [67]:
df3 = pbf.groupby('Sex')
df3.head()

,1st Go Place,Horse,Age,Color,Sex,Sire,Dam,Dam's Sire,Rider,1st Go,2nd Go Place,2nd Go,1stGoD,2ndGoD,Money_Winner,1D_Money_Winner,#
0,1.0,French Goodbye,2014.0,Palomino,Gelding,THE GOODBYE LANE,Two Fools Gold,Frenchmans Guy,Vauna Walker,16.567,5.0,16.936,1,1,1.0,1.0,NaN
1,2.0,Jets Letters Ta Abby,2014.0,Sorrel,Gelding,BLAZIN JETOLENA,Real Famous Abby,Dash Ta Fame,Ashley Schafer,16.845,7.0,16.997,1,1,1.0,1.0,NaN
2,3.0,Hes Furious,2014.0,Gelding,Gelding,THE GOODBYE LANE,Furyx,Calyx,Timi Lickley,16.988,8.0,16.998,1,1,1.0,1.0,NaN
3,4.0,Jets Heart N Soul,2014.0,Chestnut,Gelding,BLAZIN JETOLENA,Mistys Dash of Fame,Dash Ta Fame,Jolene Montgomery,16.988,2.0,16.818,1,1,1.0,1.0,NaN
4,5.0,Our First Goodbye,2013.0,Brown,Mare,THE GOODBYE LANE,Jesse James Agent,Jesse James Jr,Marcie Wilson,17.005,66.0,22.455,1,2,1.0,1.0,NaN
5,6.0,High Cotton Lane,2014.0,Chestnut,Stallion,THE GOODBYE LANE,HMB Soon Tobe Famous,Dash Ta Fame,Paige Jones,17.021,3.0,16.829,1,1,1.0,1.0,NaN
6,7.0,NB Ladybird,2014.0,Gray,Mare,SHAWNE BUG LEO,TR Dashing Badger,Mr Illuminator,Kenna Squires,17.050,11.0,17.087,1,1,1.0,1.0,NaN
7,8.0,Streakin French Star,2014.0,Buckskin,Gelding,STREAKIN BOON DOX,French Stars,Frenchmans Guy,Ashley Schafer,17.091,14.0,17.166,1,1,1.0,1.0,NaN
8,9.0,Positively Streakin,2014.0,Bay Roan,Mare,STREAKIN BOON DOX,Positively A Coyote,Coyote Colonel,Ashley Schafer,17.117,15.0,17.208,1,1,1.0,1.0,NaN
9,10.0,Mr Elegant Shawne,2014.0,Sorrel,Mare,SHAWNE BUG LEO,Elegant Seis,Tres Seis,Jacque Woolman,17.153,10.0,17.076,1,1,1.0,1.0,NaN


In [72]:
sexcount = pbf.groupby(['Sex']).sum()
sexdata = pbf.groupby(['Sex']).mean()

In [74]:
sexcount.head()
display(sexdata.head())

,1st Go Place,Age,1st Go,2nd Go Place,2nd Go,1stGoD,2ndGoD,Money_Winner,1D_Money_Winner
Sex,,,,,,,,,
Gelding,85.245714,2014.750000,102.829074,88.857143,119.784434,1.681818,1.715909,0.400000,0.300000
Mare,86.120536,2014.736607,75.918576,83.718750,93.423732,1.745536,1.674107,0.425000,0.225000
Stallion,86.093750,2014.750000,79.816906,78.968750,19.128906,1.750000,1.718750,0.428571,0.142857
